In [44]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import numpy as np

In [45]:
df = pd.read_excel('data/data.xlsx')
df

/home/uncookie/anaconda3/envs/main/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,nominal,data,curs,cdx
0,1,2023-12-23,91.9389,Доллар США
1,1,2023-12-22,91.7062,Доллар США
2,1,2023-12-21,90.4056,Доллар США
3,1,2023-12-20,90.0870,Доллар США
4,1,2023-12-19,90.4162,Доллар США
...,...,...,...,...
7199,1,1992-12-16,418.0000,Доллар США
7200,1,1992-12-11,419.0000,Доллар США
7201,1,1992-12-09,419.0000,Доллар США
7202,1,1992-12-04,398.0000,Доллар США


In [46]:
df.drop(columns=['nominal', 'cdx'])

,data,curs
0,2023-12-23,91.9389
1,2023-12-22,91.7062
2,2023-12-21,90.4056
3,2023-12-20,90.0870
4,2023-12-19,90.4162
...,...,...
7199,1992-12-16,418.0000
7200,1992-12-11,419.0000
7201,1992-12-09,419.0000
7202,1992-12-04,398.0000


In [71]:
dates = pd.to_datetime(df['data'])
rates = df['curs'].values

dates_numeric = np.array((dates - dates.min()).dt.days, dtype=np.float32)

scaler_dates = MinMaxScaler()
dates_scaled = scaler_dates.fit_transform(dates_numeric.reshape(-1, 1))

scaler_rates = MinMaxScaler()
rates_scaled = scaler_rates.fit_transform(rates.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(dates_scaled, rates_scaled, test_size=0.2, random_state=42)


In [72]:
print(dates_scaled)

[[1.0000000e+00]
 [9.9991184e-01]
 [9.9982369e-01]
 ...
 [6.1712071e-04]
 [1.7632020e-04]
 [0.0000000e+00]]


In [73]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=1))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

In [74]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
181/181 [==============================] - 3s 11ms/step - loss: 0.0472 - val_loss: 0.0399
Epoch 2/20
181/181 [==============================] - 2s 9ms/step - loss: 0.0345 - val_loss: 0.0284
Epoch 3/20
181/181 [==============================] - 2s 10ms/step - loss: 0.0241 - val_loss: 0.0212
Epoch 4/20
181/181 [==============================] - 2s 10ms/step - loss: 0.0187 - val_loss: 0.0164
Epoch 5/20
181/181 [==============================] - 2s 10ms/step - loss: 0.0169 - val_loss: 0.0148
Epoch 6/20
181/181 [==============================] - 2s 10ms/step - loss: 0.0151 - val_loss: 0.0135
Epoch 7/20
181/181 [==============================] - 2s 9ms/step - loss: 0.0139 - val_loss: 0.0124
Epoch 8/20
181/181 [==============================] - 2s 9ms/step - loss: 0.0127 - val_loss: 0.0108
Epoch 9/20
181/181 [==============================] - 2s 11ms/step - loss: 0.0116 - val_loss: 0.0094
Epoch 10/20
181/181 [==============================] - 2s 9ms/step - loss: 0.0097 - val_loss: 

In [77]:
new_date = pd.to_datetime('2021-12-12')  
new_date_numeric = (new_date - dates.min()).days
new_date_scaled = scaler_dates.transform([[new_date_numeric]])  
predicted_rate_scaled = model.predict(new_date_scaled)
predicted_rate = scaler_rates.inverse_transform(predicted_rate_scaled)


1/1 [==============================] - 0s 93ms/step


In [78]:
predicted_rate

array([[123.75592]], dtype=float32)